In [53]:
import pandas as pd
import numpy as np
from IPython.display import display

votes = [5, 5, 4, 6]
candidates = ["A", "B", "C", "D"]

matrix = [
    ["A", "C", "B", "C"],
    ["B", "A", "C", "B"],
    ["C", "B", "D", "D"],
    ["D", "D", "A", "A"],
]

print(votes)
print(candidates)
print(matrix)

[5, 5, 4, 6]
['A', 'B', 'C', 'D']
[['A', 'C', 'B', 'C'], ['B', 'A', 'C', 'B'], ['C', 'B', 'D', 'D'], ['D', 'D', 'A', 'A']]


## Правило абсолютної більшості

In [54]:
def getRelativeResults(m, votes, candidates):
    relativeResults = {candidate: 0 for candidate in candidates}
    for i in range(len(m[0])):
        key = m[0][i]
        if key in relativeResults:
            relativeResults[key] += votes[i]

    return relativeResults


def getAbsoluteWinner(m, votes, candidates, printResults=True):
    relativeResults = getRelativeResults(m, votes, candidates)
    if printResults:
        display(pd.DataFrame(relativeResults, index=["First: "]))

    return max(relativeResults, key = relativeResults.get)


absoluteWinner = getAbsoluteWinner(matrix, votes, candidates)

print("Winner: ", absoluteWinner)

,A,B,C,D
First:,5,4,11,0


Winner:  C


## Правило відносної більшості

In [55]:
def getRelative(m, votes, candidates, printResults=True):
    relativeResults = getRelativeResults(m, votes, candidates)
    if printResults:
        display(pd.DataFrame(relativeResults, index=["First: "]))

    return max(relativeResults, key=relativeResults.get)


relative_winner = getRelative(matrix, votes, candidates)

print("Winner: ", relative_winner)

,A,B,C,D
First:,5,4,11,0


Winner:  C


## Правило Борда

In [56]:
def getBorda(m, votes, candidates, printResults=True):
    S = [len(m) - i - 1 for i in range(len(m))]

    if printResults:
        display(pd.DataFrame(m, index=S, columns=votes))

    results = {candidate: 0 for candidate in candidates}

    for score, i in enumerate(S):
        for j in range(len(m[i])):
            if m[i][j] in candidates:
                results[m[i][j]] += votes[j] * score

    if printResults:
        display(pd.DataFrame(results, index=["Results"]))

    return max(results, key=results.get)


borda_winner = getBorda(matrix, votes, candidates)

print("Winner: ", borda_winner)

,5,5,4,6
3,A,C,B,C
2,B,A,C,B
1,C,B,D,D
0,D,D,A,A


,A,B,C,D
Results,25,39,46,10


Winner:  C


## Правило Кондорсе

In [57]:
def getCondorcet(m, votes, candidates):
    num_candidates = len(candidates)

    pairwise_matrix = [[0] * num_candidates for _ in range(num_candidates)]

    for i in range(num_candidates): 
        for j in range(num_candidates):
            res = getRelativeResults(m, votes, candidates)
            k_i = res[candidates[i]]
            k_j = res[candidates[j]]
            if k_i >= k_j:  
                pairwise_matrix[i][j] = True
            elif k_i < k_j:
                pairwise_matrix[i][j] = False

    winner = None
    for i in range(num_candidates): 
        for j in range(num_candidates): 
            if np.all(pairwise_matrix[i], 0) and i != j:
                winner = candidates[i]

    return winner, pairwise_matrix

winner, pairwise_matrix = getCondorcet(matrix, votes, candidates)
display(pd.DataFrame(pairwise_matrix, index = candidates, columns = candidates))

print("Winner: ", winner)

,A,B,C,D
A,True,True,False,True
B,False,True,False,True
C,True,True,True,True
D,False,False,False,True


Winner:  C


## Правило Копланда

In [58]:
def getCopeland(m, votes, candidates):
    num_candidates = len(candidates)
    copeland_scores = [0] * num_candidates

    for i in range(num_candidates): 
        for j in range(num_candidates):  
            if i != j:  
                wins = 0 
                res = getRelativeResults(m, votes, candidates)
                k_i = res[candidates[i]]
                k_j = res[candidates[j]]
                if k_i > k_j:  
                    wins += 1
                elif k_i < k_j: 
                    wins -= 1
                copeland_scores[i] += wins

    max_score = max(copeland_scores)
    winner = candidates[copeland_scores.index(max_score)]

    return winner, copeland_scores

copeland_winner, copeland_scores = getCopeland(matrix, votes, candidates)

display(pd.DataFrame(copeland_scores, index=candidates, columns=["Бали Копленда"]))
print("Winner: ", copeland_winner)

,Бали Копленда
A,1
B,-1
C,3
D,-3


Winner:  C


## Правило Сімпсона

In [59]:
def getSimpson(m, votes, candidates):
    num_candidates = len(candidates)
    num_votes = len(votes)

    losses = [0] * num_candidates

    for i in range(num_candidates): 
        for j in range(num_candidates): 
            if i != j: 
                wins = 0  
                for k in range(num_votes): 
                    k_i = m[k].index(candidates[i]) if candidates[i] in m[k] else 0
                    k_j = m[k].index(candidates[j]) if candidates[j] in m[k] else 0
                    if k_i > k_j:  
                        wins += votes[k]
                    elif k_i < k_j:  
                        wins -= votes[k]
                if wins < 0:  
                    losses[i] += 1

    min_losses = min(losses)
    winner = candidates[losses.index(min_losses)]

    return winner, losses

simpson_winner, simpson_losses = getSimpson(matrix, votes, candidates)
display(pd.DataFrame(simpson_losses, index = candidates))

print("Winner: ", simpson_winner)

,0
A,1
B,1
C,1
D,3


Winner:  A
